In [2]:
from queue import PriorityQueue
def cbs_fixnum(rtree, eps, n_partition):
    mbr = rtree.bounds
    partition_list = []
    queue = PriorityQueue()
    cost = _get_cost(mbr, rtree.count(mbr))
    queue.put((-cost,mbr) )
    while queue.qsize() < n_partition:
        (score, br) = queue.get()
        subbrs = _cost_base_split(rtree, br, eps)
        for subbr in subbrs:
            cost = _get_cost(subbr, rtree.count(subbr))
            queue.put((-cost,subbr))
    while queue.qsize()>0:
        (score, br) = queue.get()
        partition_list.append(br)
    return partition_list

def rbs_fixnum(rtree, eps, n_partition):
    mbr = rtree.bounds
    partition_list = []
    queue = PriorityQueue()
    queue.put((-rtree.count(mbr),mbr) )
    while queue.qsize() < n_partition:
        (score, br) = queue.get()
        subbrs = _reduced_boundary_split(rtree, br, eps)
        for subbr in subbrs:

            queue.put((-rtree.count(subbr),subbr))
    while queue.qsize()>0:
        (score, br) = queue.get()
        partition_list.append(br)
    return partition_list

def rtree_partition(dataset, n_patitions, eps,mtd='cbs'):
    """
    params:parameters needed for different spliting method; for cost-base method params=('cbs', max_cost);
    for reduced-boundary method params = ('rbs', max_points)
    """    
    idx = construct_rtree_index(dataset)
    # #split test
    if mtd=='cbs':
        partitioned = cbs_fixnum(idx, eps, n_patitions)
    else:
        partitioned = rbs_fixnum(idx, eps, n_patitions)

    indexed_data = []
    id_ptt = 0
    for boundary in partitioned:    
        (left, bot, right, top) = boundary
        for id_pts in idx.intersection((left-eps, bot-eps, right+eps, top+eps )):
            indexed_data.append([id_ptt, id_pts])
        id_ptt+=1

    res = sc.parallelize(indexed_data).groupByKey().map(lambda x: [x[0], list(x[1])])
    return res


def parallel_dbscan(dataset, eps, n_partitions, rtree = False,rtree_mtd='cbs'):
    if rtree:
        partitioned_rdd = rtree_partition(dataset, n_partitions, eps, mtd=rtree_mtd)
    else:
        partitioned_rdd = partition(dataset, n_partitions, eps)
    local_tags = partitioned_rdd.mapValues(lambda x: local_dbscan(x)).collect()
    result_tags = merge(local_tags, dataset)
    
    return result_tags